In [ ]:
!pip install --upgrade coremltools transformers
#might not need transformers datasets torch
!pip install transformers datasets torch
import torch
import coremltools as ct
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"  # Use a DistilBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# Create sample input
sample_text = "This is a sample text for tracing."
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)

# Define a class to wrap the model (if needed)
class ModelWrapper(torch.nn.Module):
def __init__(self, model):
super(ModelWrapper, self).__init__()
self.model = model

def forward(self, input_ids, attention_mask):
outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
return outputs.logits  # Adjust output based on DistilBERT's structure

# Create an instance of the wrapper class
model_wrapper = ModelWrapper(model)


# Instead of scripting, trace the model
traced_model = torch.jit.trace(model_wrapper, (inputs["input_ids"], inputs["attention_mask"]))

# Save the traced model
traced_model.save("distilbert_traced.pt")

mlmodel = ct.convert(
"distilbert_traced.pt",
source="pytorch",
inputs=[
ct.TensorType(name="input_ids", shape=inputs["input_ids"].shape),
ct.TensorType(name="attention_mask", shape=inputs["attention_mask"].shape)
],
minimum_deployment_target=ct.target.iOS16,
convert_to="mlprogram"
)


# Save the model
mlmodel.save("distilbert_classification.mlpackage")

!zip -r distilbert_classification.zip distilbert_classification.mlpackage

# Download using Colab's files utility
from google.colab import files
files.download('distilbert_classification.zip')